In [130]:
from agavepy.agave import Agave
from warnings import filterwarnings
import pprint 
import json, os
filterwarnings('ignore')
pp = pprint.PrettyPrinter(indent=2,depth=6, width=60)

In [42]:
os.environ.update({"AGAVE_PASSWORD":"PASSWORD","AGAVE_BASEURL": "https://public.agaveapi.co"})

In [43]:
api = Agave(username = os.environ.get('AGAVE_USERNAME'),
            password = os.environ.get('AGAVE_PASSWORD'),
            api_server = os.environ.get('AGAVE_BASEURL'),
            clientName="jupyter_examples")

In [46]:
client = api.clients.create(body={'clientName':"jupyter_examples"})

In [ ]:
api.token.create()

In [ ]:
api.profiles.listByUsername(username="me")

# Basic job listings

We can fetch a standard job listing using the empty `agave.jobs.list()` method call. A list of dict objects will be returned

In [172]:
jobs = api.jobs.list()
print("Type of response is %s"%(type(jobs)))
print("Each list element is a %s"%(type(jobs[0])))

Type of response is <type 'list'>
Each list element is a <class 'agavepy.agave.AttrDict'>


Each dict represents a job and contains summary info about the job.

In [185]:
jobs[0]

{u'_links': {u'archiveData': {u'href': u'https://public.agaveapi.co/files/v2/listings/system/data.agaveapi.co/dooley/archive/jobs/job-4218907132382604825-242ac11c-0001-007'},
  u'self': {u'href': u'https://public.agaveapi.co/jobs/v2/4218907132382604825-242ac11c-0001-007'}},
 u'appId': u'cloud-runner-0.1.0u1',
 u'created': u'2017-12-05T14:20:44.000-06:00',
 u'endTime': datetime.datetime(2017, 12, 5, 14, 21, 53, tzinfo=tzoffset(None, -21600)),
 u'executionSystem': u'docker.tacc.utexas.edu',
 u'id': u'4218907132382604825-242ac11c-0001-007',
 u'name': u'dooley-cloud-runner-0.1.0u1-1512504690',
 u'owner': u'dooley',
 u'startTime': datetime.datetime(2017, 12, 5, 14, 21, 33, tzinfo=tzoffset(None, -21600)),
 u'status': u'FINISHED'}

## Pagination

Pagination is supported and, in fact, enforced by default. Page size defaults to 100 results. You can specify an alternate number of results with the `limit` parameter.

In [193]:
jobs = api.jobs.list(search={"owner.like":"*"}, filter="id")
print ("The default page size in this tenant is: %d"%(len(jobs)))

The default page size in this tenant is: 72


The max page size varies by tenant, but is usually no more than 500 for the jobs api. You can test this out (provided sufficient data) simply by requsting a large result set and checking the actual number returned.

In [148]:
jobs = api.jobs.list(search={"owner.like":"*"}, filter="id", limit=9999999)
print ("The max page size in this tenant is: %d"%(len(jobs)))

The max page size in this tenant is: 300


No pagination parameters are included in the response at this time, so you need to paginate until you either get an empty response, or get back fewer than the number of results you requested with the `limit` parameter.

In [164]:
pageSize = 50
pageCount = 0

totalJobs = 0
jobs = api.jobs.list(search={"name.eq":"pysdk_testsuite"}, offset=pageCount, filter="id", limit=pageSize)
while len(jobs) != 0:
    pageCount = pageCount + 1
    totalJobs = totalJobs + len(jobs)
    print ("Page %d query returned %d jobs"%(pageCount, len(jobs)))
    jobs = api.jobs.list(search={"name.eq":"pysdk_testsuite"}, filter="id", offset=(pageCount*pageSize), limit=pageSize)

if len(jobs) > 0:
    totalJobs = totalJobs + len(jobs)
    pageCount = pageCount + 1
    print ("Page %d query returned %d jobs"%(pageCount, len(jobs)))

print ("There were %d total jobs returned across %d pages"%(totalJobs, pageCount))

Page 1 query returned 50 jobs
Page 2 query returned 50 jobs
Page 3 query returned 50 jobs
Page 4 query returned 50 jobs
Page 5 query returned 50 jobs
Page 6 query returned 22 jobs
There were 272 total jobs returned across 6 pages


## Response filtering

Notice in all the listing responses thus far, a subset of the full job details is returned. This is a convention to reduce response sizes. Agave provides a `filter` parameter enabling you to customize the response objects by specifying a comma-delimited list of fields you would like in the response.

In the following example, we request just the job id, app id, and job status in the response.

In [200]:
jobs = api.jobs.list(filter="id,appId,status", limit=1)
#jobs
print json.dumps(jobs, sort_keys=True, indent=4)

[
    {
        "appId": "cloud-runner-0.1.0u1", 
        "id": "4218907132382604825-242ac11c-0001-007", 
        "status": "FINISHED"
    }
]


Specifying a nested object in the filter will return just that part of the object

In [201]:
jobs = api.jobs.list(filter="id,_links.executionSystem", limit=1)
#jobs
print json.dumps(jobs, sort_keys=True, indent=4)

[
    {
        "_links": {
            "executionSystem": {
                "href": "https://public.agaveapi.co/systems/v2/docker.tacc.utexas.edu"
            }
        }, 
        "id": "4218907132382604825-242ac11c-0001-007"
    }
]


Specifying multiple nested objects will honor the orignial structure

In [203]:
jobs = api.jobs.list(filter="appId,parameters.xmax,parameters.xmin", limit=1)
#jobs
print json.dumps(jobs, sort_keys=True, indent=4)

[
    {
        "appId": "cloud-runner-0.1.0u1", 
        "parameters": {
            "xmax": null, 
            "xmin": null
        }
    }
]


We can request the full object response by using a wildcard character, `*`, in the filter

In [213]:
jobs = api.jobs.list(filter="*", limit=1)
#jobs
jobs

[{u'_links': {u'app': {u'href': u'https://public.agaveapi.co/apps/v2/cloud-runner-0.1.0u1'},
   u'archiveData': {u'href': u'https://public.agaveapi.co/files/v2/listings/system/data.agaveapi.co/dooley/archive/jobs/job-4218907132382604825-242ac11c-0001-007'},
   u'archiveSystem': {u'href': u'https://public.agaveapi.co/systems/v2/data.agaveapi.co'},
   u'executionSystem': {u'href': u'https://public.agaveapi.co/systems/v2/docker.tacc.utexas.edu'},
   u'history': {u'href': u'https://public.agaveapi.co/jobs/v2/4218907132382604825-242ac11c-0001-007/history'},
   u'metadata': {u'href': u'https://public.agaveapi.co/meta/v2/data/?q=%7B%22associationIds%22%3A%224218907132382604825-242ac11c-0001-007%22%7D'},
   u'notifications': {u'href': u'https://public.agaveapi.co/notifications/v2/?associatedUuid=4218907132382604825-242ac11c-0001-007'},
   u'owner': {u'href': u'https://public.agaveapi.co/profiles/v2/dooley'},
   u'permissions': {u'href': u'https://public.agaveapi.co/jobs/v2/4218907132382604825-

## General search syntax

We can search across nearly all of Agave's APIs using a common jsonsql syntax.**


`"<field>.<operator>" : "value"`

The filed value is any attibute of the resource in the collection you are searching. The supported operators are listed in the following table.  


| Operator | Values                     | Description                                                                                                                                                                          |
|----------|----------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| eq       | mixed                      | Matches values equal to the given search value. All comparisons are case sensitive. This cannot be used for complex object comparison.                                               |
| on       | datestring                 | Matches dates falling on the given datestring. Regardless of the precision given in the datestring, the search will look for matches from midnight to midnight on the resovled date. |
| neq      | mixed                      | Matches values not equal to the given search value. All comparisons are case sensitive. This cannot be used for complex object comparison.                                           |
| lt       | mixed                      | Matches values less than the given search value.                                                                                                                                     |
| before   | datestring                 | Matches dates falling before the given datestring. Single second precision is supported.                                                                                             |
| lte      | mixed                      | Matches values less than or equal to the given search value.                                                                                                                         |
| gt       | mixed                      | Matches values greater than the given search value.                                                                                                                                  |
| after    | datestring                 | Matches values after the given datestring.                                                                                                                                           |
| gte      | mixed                      | Matches values greater than or equal to the given search value.                                                                                                                      |
| in       | comma-separated list       | Matches values in the given comma-separated list. This is equivalent to applying the like operator to each comma-separated value .                                                   |
| nin      | comma-separated list       | Matches values not in the given comma-separated list. This is equivalent to applying the nlike operator to each comma-separated value .                                              |
| like     | string                     | Matches values similar to the given search term. Wildcards (*) may be used to perform partial matches.                                                                               |
| nlike    | string                     | Matches values different from the given search term. Wildcards (*) may be used to perform partial matches.                                                                           |
| between  | comma-separated datestring | Matches dates falling within the given range. Single second precision is supported at either end of the range.                                                                       |

> ** _(Metadata, profiles, and clients, are exceptions. Metadata uses mongodb query language. Profiles supports search by name an email only. Clients does not support seach)._ 


By default, if no operator is specified, an equality operator is implied, thus the following two queries are equivalent

In [215]:
searchByImplicitOperator = api.jobs.list(search={"id":"9059371012861268456-242ac11c-0001-007"}, filter="*")
searchByExplicitOperator = api.jobs.list(search={"id.eq":"9059371012861268456-242ac11c-0001-007"}, filter="*")
#Any of the resource fields are valid search fields.
assert(searchByImplicitOperator[0]['id'] == searchByExplicitOperator[0]['id'])
assert(len(searchByImplicitOperator) == len(searchByExplicitOperator))

Valid fields are any of the resource's attributes. For a job resource, the following fields are valid:

| Field name        | Description                                                                                                                                                      |
|-------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| appId             | The id of the app run by the job.                                                                                                                                |
| archive           | Boolean flag stating whether the job output  was archived.                                                                                                       |
| archivePath       | Path on the archive system where the job output was archived if the archive flag was true.                                                                       |
| archiveSystem     | The id of the system where the job output was archived if the archive flag was true.                                                                             |
| batchQueue        | The system queue in which the job ran.                                                                                                                           |
| created           | The date the job request was made. You may specify using structured or free form timeframes such as 'yesterday' or '-3 hours'.                                   |
| endTime           | The date the job finished running. You may specify using structured or free form timeframes such as 'yesterday' or '-3 hours'.                                   |
| executionSystem   | The execution system where the job ran.                                                                                                                          |
| id                | The id of the job.                                                                                                                                               |
| inputs            | The job inputs. Note, this is currently a full text match.                                                                                                       |
| lastUpdated       | The date the job was last updated with a status change. Times may be specified in structured or free form timeframes such as 'yesterday' or '-3 days'.           |
| localId           | The local job id of the job on the execution system.                                                                                                             |
| maxRunTime        | The maximum run time of the job in HH:mm:ss format.                                                                                                              |
| memoryPerNode     | The memory requested by the job specified in GB.                                                                                                                 |
| name              | The name of the job.                                                                                                                                             |
| nodeCount         | The number of nodes requested for the job.                                                                                                                       |
| owner             | The user who submitted the job                                                                                                                                   |
| outputPath        | The remote work directory path of the job.                                                                                                                       |
| parameters        | The job parameters. Note, this is currently a full text match.                                                                                                   |
| processorsPerNode | The number of processors per node requested by the job.                                                                                                          |
| retries           | The number of retry attempts made on this job.                                                                                                                   |
| runtime           | The duration of the job in seconds.                                                                                                                              |
| startTime         | The date the job began running. Results are rounded by day. You may specify using YYYY-MM-DD format or free form timeframes such as 'yesterday' or '3 days ago'. |
| status            | The job status.                                                                                                                                                  |
| submitTime        | The date the job was submitted to the remote execution system to run. You may specify using structured or free form timeframes such as 'yesterday' or '-3 days'. |
| visible           | Boolean flag indicating whether or not to show deleted jobs. Defaults to false.                                                                                  |

## Search examples

Seaching by a list of job ids

In [217]:
jobs = api.jobs.list(search={"id.in":"3541227267326864920-242ac11c-0001-007,4381740753562431000-242ac11c-0001-007,9059371012861268456-242ac11c-0001-007"}, filter="id,appId", limit=10)
#jobs
jobs

[{u'appId': u'funwave-tvd-shelob-stevenrbrandt-1.0',
  u'id': u'9059371012861268456-242ac11c-0001-007'},
 {u'appId': u'funwave-tvd-shelob-stevenrbrandt-1.0',
  u'id': u'4381740753562431000-242ac11c-0001-007'},
 {u'appId': u'funwave-tvd-shelob-stevenrbrandt-1.0',
  u'id': u'3541227267326864920-242ac11c-0001-007'}]

Job submitted within a machine-readable date range

In [220]:
jobs = api.jobs.list(search={'created.between':'2016-06-01,2016-06-30'},filter='created,id',limit=10)
jobs

[{u'created': u'2016-06-16T15:30:07.000-05:00',
  u'id': u'2098132481776620006-242ac114-0001-007'},
 {u'created': u'2016-06-28T11:34:05.000-05:00',
  u'id': u'2390397378563543526-242ac114-0001-007'},
 {u'created': u'2016-06-28T11:27:52.000-05:00',
  u'id': u'4791057735071109606-242ac114-0001-007'},
 {u'created': u'2016-06-24T18:12:31.000-05:00',
  u'id': u'3987056349882552806-242ac114-0001-007'},
 {u'created': u'2016-06-24T19:12:26.000-05:00',
  u'id': u'3801341964003512806-242ac114-0001-007'},
 {u'created': u'2016-06-22T23:39:41.000-05:00',
  u'id': u'7580332640414068250-242ac114-0001-007'},
 {u'created': u'2016-06-23T00:39:37.000-05:00',
  u'id': u'7722753755949428250-242ac114-0001-007'},
 {u'created': u'2016-06-20T16:08:21.000-05:00',
  u'id': u'3222794955206946330-242ac114-0001-007'},
 {u'created': u'2016-06-18T15:06:15.000-05:00',
  u'id': u'3337347505454247450-242ac114-0001-007'},
 {u'created': u'2016-06-18T02:47:20.000-05:00',
  u'id': u'7495728450536861210-242ac114-0001-007'}]

Job submitted within a human-readable date range

In [237]:
jobs = api.jobs.list(search={'created.between':'last week,today'},filter='created,id',limit=10)
jobs

[{u'created': u'2018-02-02T12:10:50.000-06:00',
  u'id': u'7292002072357245416-242ac11c-0001-007'},
 {u'created': u'2018-02-02T12:59:10.000-06:00',
  u'id': u'2734178173637562856-242ac11c-0001-007'},
 {u'created': u'2018-02-01T22:35:53.000-06:00',
  u'id': u'8560316566115053080-242ac11c-0001-007'},
 {u'created': u'2018-02-01T21:55:56.000-06:00',
  u'id': u'824291154109656600-242ac11c-0001-007'},
 {u'created': u'2018-01-31T21:08:17.000-06:00',
  u'id': u'2379744003332313576-242ac11c-0001-007'},
 {u'created': u'2018-01-31T14:55:57.000-06:00',
  u'id': u'2433772080572469736-242ac11c-0001-007'}]

Job submitted within a human-readable timeframe range

In [239]:
jobs = api.jobs.list(search={'created.after':'1 hour ago'},filter='created,id',limit=10)
jobs

[{u'created': u'2018-02-06T16:08:57.000-06:00',
  u'id': u'5974073971606950376-242ac11c-0001-007'},
 {u'created': u'2018-02-06T13:21:58.000-06:00',
  u'id': u'80722638825975320-242ac11c-0001-007'}]

Jobs whose name matches "jobc" anywhere

In [234]:
jobs = api.jobs.list(search={'name.like':'*command*'},filter='name,id',limit=5)
jobs

[{u'id': u'6379017166622101991-242ac11b-0001-007', u'name': u'fork-command-1'},
 {u'id': u'2225053646949781991-242ac11b-0001-007', u'name': u'fork-command-1'},
 {u'id': u'1210625321307541991-242ac11b-0001-007', u'name': u'fork-command-1'},
 {u'id': u'4003379076690603545-242ac11b-0001-007', u'name': u'fork-command-1'},
 {u'id': u'4971421755536043545-242ac11b-0001-007', u'name': u'fork-command-1'}]

Jobs whose name starts with "agave"

In [235]:
jobs = api.jobs.list(search={'name.like':'agave*'},filter='name,id',limit=5)
jobs

[{u'id': u'2433772080572469736-242ac11c-0001-007', u'name': u'agavetest'},
 {u'id': u'3323381436709793305-242ac11c-0001-007', u'name': u'agavetest'},
 {u'id': u'7129947935458782745-242ac11c-0001-007', u'name': u'agavetest'},
 {u'id': u'6670355098705456665-242ac11c-0001-007', u'name': u'agavetest'},
 {u'id': u'7079966129724976665-242ac11c-0001-007', u'name': u'agavetest'}]

Jobs whose name ends with "jobc-1" anywhere

In [236]:
jobs = api.jobs.list(search={'name.like':'*jobc-1'},filter='name,id',limit=5)
jobs

[{u'id': u'5974073971606950376-242ac11c-0001-007', u'name': u'test-jobc-1'},
 {u'id': u'9059371012861268456-242ac11c-0001-007', u'name': u'test-jobc-1'},
 {u'id': u'4381740753562431000-242ac11c-0001-007', u'name': u'test-jobc-1'},
 {u'id': u'3541227267326864920-242ac11c-0001-007', u'name': u'test-jobc-1'},
 {u'id': u'925615411246132760-242ac11c-0001-007', u'name': u'test-jobc-1'}]

Jobs with an active status

In [240]:
jobs = api.jobs.list(search={'status.nin':'STOPPED,FINISHED,FAILED,PAUSED'},filter='status,id',limit=5)
jobs

[{u'id': u'5974073971606950376-242ac11c-0001-007', u'status': u'QUEUED'},
 {u'id': u'80722638825975320-242ac11c-0001-007', u'status': u'RUNNING'},
 {u'id': u'3541227267326864920-242ac11c-0001-007', u'status': u'SUBMITTING'},
 {u'id': u'925615411246132760-242ac11c-0001-007', u'status': u'SUBMITTING'},
 {u'id': u'4771930423505972760-242ac11c-0001-007', u'status': u'SUBMITTING'}]

Failed jobs with core count greater than or equal to 8 that running in a queue named "normal" or "long" queues

In [241]:
jobs = api.jobs.list(search={'processorsPerNode.gte': 8, 'batchQueue.in':'normal,long', 'status.eq': 'FAILED'}, filter='status,batchQueue,processorsPerNode,executionSystem',limit=5)
jobs

[{u'batchQueue': u'normal',
  u'executionSystem': u'stampede-fonner',
  u'processorsPerNode': 16,
  u'status': u'FINISHED'},
 {u'batchQueue': u'normal',
  u'executionSystem': u'stampede-fonner',
  u'processorsPerNode': 16,
  u'status': u'FINISHED'},
 {u'batchQueue': u'normal',
  u'executionSystem': u'stampede-fonner',
  u'processorsPerNode': 16,
  u'status': u'FAILED'},
 {u'batchQueue': u'normal',
  u'executionSystem': u'stampede-fonner',
  u'processorsPerNode': 16,
  u'status': u'FINISHED'},
 {u'batchQueue': u'normal',
  u'executionSystem': u'stampede-fonner',
  u'processorsPerNode': 16,
  u'status': u'FAILED'}]

Job with a parameter name "xmax"

In [244]:
jobs = api.jobs.list(search={"parameters.like": '*xmax*'}, filter="id,parameters,appId", limit=10)
jobs

[{u'appId': u'drawgau-shelob-stevenrbrandt-1.0',
  u'id': u'5974073971606950376-242ac11c-0001-007',
  u'parameters': {u'dx': u'0.1', u'xmax': 10, u'xmin': u'-1'}},
 {u'appId': u'funwave-tvd-shelob-stevenrbrandt-1.0',
  u'id': u'9059371012861268456-242ac11c-0001-007',
  u'parameters': {u'dx': u'0.1', u'xmax': 10, u'xmin': u'-1'}},
 {u'appId': u'funwave-tvd-shelob-stevenrbrandt-1.0',
  u'id': u'4381740753562431000-242ac11c-0001-007',
  u'parameters': {u'dx': u'0.1', u'xmax': 2, u'xmin': u'-1'}},
 {u'appId': u'funwave-tvd-shelob-stevenrbrandt-1.0',
  u'id': u'3541227267326864920-242ac11c-0001-007',
  u'parameters': {u'dx': u'0.1', u'xmax': 2, u'xmin': u'-1'}},
 {u'appId': u'funwave-tvd-shelob-stevenrbrandt-1.0',
  u'id': u'925615411246132760-242ac11c-0001-007',
  u'parameters': {u'dx': u'0.1', u'xmax': 2, u'xmin': u'-1'}},
 {u'appId': u'funwave-tvd-shelob-stevenrbrandt-1.0',
  u'id': u'4771930423505972760-242ac11c-0001-007',
  u'parameters': {u'dx': u'0.1', u'xmax': 2, u'xmin': u'-1'}}]

Jobs with a parameter named compress_output and value True

In [255]:
jobs = api.jobs.list(search={"parameters.like": '*"compress_output":true*'}, filter="id,parameters,appId", limit=10)
jobs

[{u'appId': u'training034-training034-funwave-1.0',
  u'id': u'6890774133532135911-242ac11b-0001-007',
  u'parameters': {u'AMP': u'4.0', u'WID': u'20.0', u'compress_output': True}},
 {u'appId': u'training046-training046-funwave-1.0',
  u'id': u'7790569782044135911-242ac11b-0001-007',
  u'parameters': {u'AMP': u'4.0', u'WID': u'20.0', u'compress_output': True}},
 {u'appId': u'training039-training039-funwave-1.0',
  u'id': u'6371340788753895911-242ac11b-0001-007',
  u'parameters': {u'AMP': u'4.0', u'WID': u'20.0', u'compress_output': True}},
 {u'appId': u'stevenrbrandt-sandbox-funwave-1.0',
  u'id': u'4436216878533373465-242ac11b-0001-007',
  u'parameters': {u'AMP': u'4.0', u'WID': u'20.0', u'compress_output': True}},
 {u'appId': u'training001-training001-funwave-1.0',
  u'id': u'8026888640813994471-242ac11b-0001-007',
  u'parameters': {u'AMP': u'4.0', u'WID': u'20.0', u'compress_output': True}}]

In [ ]:
Jobs with a parameter named AMP and value of 4.0

In [272]:
terms={"parameters.like": '*"AMP":"4.0"*'}
jobs = api.jobs.list(search=terms, filter="id,parameters,appId", limit=10)
jobs

[{u'appId': u'training034-training034-funwave-1.0',
  u'id': u'6890774133532135911-242ac11b-0001-007',
  u'parameters': {u'AMP': u'4.0', u'WID': u'20.0', u'compress_output': True}},
 {u'appId': u'training046-training046-funwave-1.0',
  u'id': u'7790569782044135911-242ac11b-0001-007',
  u'parameters': {u'AMP': u'4.0', u'WID': u'20.0', u'compress_output': True}},
 {u'appId': u'training039-training039-funwave-1.0',
  u'id': u'6371340788753895911-242ac11b-0001-007',
  u'parameters': {u'AMP': u'4.0', u'WID': u'20.0', u'compress_output': True}},
 {u'appId': u'stevenrbrandt-sandbox-funwave-1.0',
  u'id': u'4436216878533373465-242ac11b-0001-007',
  u'parameters': {u'AMP': u'4.0', u'WID': u'20.0', u'compress_output': True}},
 {u'appId': u'training001-training001-funwave-1.0',
  u'id': u'8026888640813994471-242ac11b-0001-007',
  u'parameters': {u'AMP': u'4.0', u'WID': u'20.0', u'compress_output': True}}]

Jobs run with a "command" parameter equal to "jellyfish count" and the term "singularity" in the "commandArgs" parameter

In [274]:
terms={"parameters.like": '*"command":"jellyfish count"*"commandArgs":"*singularity*"*'}
jobs = api.jobs.list(search=terms, filter="id,parameters,appId", limit=1000)
jobs

[{u'appId': u'jfonner-run-singularity-4.2.3',
  u'id': u'5666214359159476711-242ac11c-0001-007',
  u'parameters': {u'command': u'jellyfish count',
   u'commandArgs': u'-m 21 -s 100M -t 10 -C -o run-singularity.jf sample.fasta',
   u'unpackImage': True}},
 {u'appId': u'jfonner-run-singularity-4.2.3',
  u'id': u'6977370361690722791-242ac11c-0001-007',
  u'parameters': {u'command': u'jellyfish count',
   u'commandArgs': u'-m 21 -s 100M -t 10 -C -o run-singularity.jf sample.fasta',
   u'unpackImage': True}},
 {u'appId': u'jfonner-run-singularity-4.2.3',
  u'id': u'7033365121638928871-242ac11c-0001-007',
  u'parameters': {u'command': u'jellyfish count',
   u'commandArgs': u'-m 21 -s 100M -t 10 -C -o run-singularity.jf sample.fasta',
   u'unpackImage': True}},
 {u'appId': u'jfonner-run-singularity-4.2.3',
  u'id': u'1645389394042416665-242ac11c-0001-007',
  u'parameters': {u'command': u'jellyfish count',
   u'commandArgs': u'-m 21 -s 100M -t 10 -C -o run-singularity.jf sample.fasta',
   u'un

Jobs submitted to a system with "stampede" in the id that failed at least once before completing

In [275]:
terms={"executionSystem.like": '*stampede*', 'retries.gt': 1}
jobs = api.jobs.list(search=terms, filter="id,executionSystem,retries,batchQueue,appId", limit=10)
jobs

[{u'appId': u'jfonner-run-singularity-4.2.3',
  u'batchQueue': u'normal',
  u'executionSystem': u'stampede-fonner',
  u'id': u'8457952584876888551-242ac11c-0001-007',
  u'retries': 2},
 {u'appId': u'jfonner-run-singularity-4.2.3',
  u'batchQueue': u'normal',
  u'executionSystem': u'stampede-fonner',
  u'id': u'6977370361690722791-242ac11c-0001-007',
  u'retries': 2},
 {u'appId': u'jfonner-run-singularity-4.2.3',
  u'batchQueue': u'normal',
  u'executionSystem': u'stampede-fonner',
  u'id': u'3265398421756187111-242ac11c-0001-007',
  u'retries': 2},
 {u'appId': u'XSEDE16-wc-train203-1.0',
  u'batchQueue': u'normal-mic',
  u'executionSystem': u'stampede-XSEDE16-train203',
  u'id': u'8168029301700816410-242ac114-0001-007',
  u'retries': 2},
 {u'appId': u'XSEDE16-wc-train203-1.0',
  u'batchQueue': u'normal-mic',
  u'executionSystem': u'stampede-XSEDE16-train203',
  u'id': u'65756910080290330-242ac114-0001-007',
  u'retries': 2},
 {u'appId': u'XSEDE16-wc-train240-1.0',
  u'batchQueue': u'no